In [466]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli, pauli_basis, SuperOp, PTM, Operator
from qiskit.circuit.library import CXGate, CZGate
from random import choice, choices
from itertools import product, permutations, cycle
from scipy.optimize import curve_fit, nnls
from matplotlib import pyplot as plt
import numpy as np
from qiskit.providers.aer.noise import NoiseModel

In [1156]:
#display a progress bar for my sanity
def progressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()

In [1164]:
#generate circuits and metadata for learning the noise in a layer of single-qubit gates
class NoiseLearningExperiment:
    def __init__(self, layer, backend, samples=64, depths = [2,4,8,16,32,64,128]):
        #initialize variables
        self.layer = layer
        self.depths = depths
        self.backend = backend
        self.samples = samples

        self.n = len(layer) #number of qubits nontrivially affected in layer
        self.num_qubits = len(backend.properties().to_dict()["qubits"])

        #parse coupling map to contain only qubits in layer
        proc_coupling = backend.configuration().coupling_map
        self.coupling_map = [(q1, q2) for q1,q2 in proc_coupling if q1 in layer and q2 in layer]

        #generate adjacency matrix, pauli strings, and transpilation dictionaries for later use
        self.adjacency_matrix = self.get_connectivity()
        self.pauli_strings = self.get_pauli_strings()
        self.raw_gates = {}
        self.transpiled_gates = {} #store dictionary of transpiled gates
        self.raw_gates, self.transpiled_gates = self._generate_gate_dict()
        self.conjugate_gates = self._generate_conjugate_gates()

    def unmapped(self, i):
        return self.layer.index(i)

    #parse coupling map to generate adjacency matrix and find the degree of each
    def get_connectivity(self):
        qubits = self.layer
        connections = self.coupling_map.copy() 
        n = self.n

        #remap qubits so that layer is sequential
        verts = [self.unmapped(qubit) for qubit in qubits]
        edges = [(self.unmapped(qubit1), self.unmapped(qubit2)) for qubit1,qubit2 in connections]

        #adjacency matrix has a 1 at i,j if i and j are connected, 0 otherwise
        adjacency_matrix = [[0 for i in verts] for j in verts] 
        for (vert1,vert2) in edges:
            adjacency_matrix[vert1][vert2] = 1
            adjacency_matrix[vert2][vert1] = 1

        return adjacency_matrix

    #The method acts as a head for the recursive sweeping procedure used to generate the pauli strings
    def get_pauli_strings(self):
        verts = range(self.n) #qubits in layer remapped in numerical order
        start_vertex = 0 #this should be chosen to be an edge qubit
        self.pauli_strings = [['I']*self.n for i in range(9)]
        visited_verts = [] #keep track of veritices for which the bases are already selected
        while(True): #if there are isolated regions _getstr bottoms out, this method restarts it
            remaining_verts = [v for v in verts if v not in visited_verts]
            if len(remaining_verts) == 0:
                self.pauli_strings = [Pauli("".join(arr[::-1])) for arr in self.pauli_strings]
                print(visited_verts)
                return self.pauli_strings 
            else: 
                self._getstr(remaining_verts[0], visited_verts)

    #recursive sweeping procedure to find pauli strings based on number of predecessor vertices
    def _getstr(self, vertex, visited_verts):
        pauli_strings = self.pauli_strings
        adjacency_matrix = self.adjacency_matrix
        
        #copied from Fig. S3 in van den Berg
        example_orderings = {"XXXYYYZZZ":"XYZXYZXYZ",
                            "XXXYYZZZY":"XYZXYZXYZ",
                            "XXYYYZZZX":"XYZXYZXYZ",
                            "XXZYYZXYZ":"XYZXZYZYX",
                            "XYZXYZXYZ":"XYZZXYYZX"}

        visited_verts.append(vertex)
        children = [i for i,e in enumerate(adjacency_matrix[vertex]) if e == 1]
        visited_children = [c for c in children if c in visited_verts]

        match len(visited_children):
            case 0:
                cycp = cycle("XYZ")
                for i,s in enumerate(pauli_strings):
                    pauli_strings[i][vertex] = next(cycp)

            case 1:
                predecessor = visited_children[0]
                #store permutation of indices so that predecessor has X,X,X,Y,Y,Y,Z,Z,Z
                reorder_list = [[] for i in range(3)]
                for i in range(9):
                    basis = pauli_strings[i][predecessor]
                    reorder_list["XYZ".index(basis)].append(i)
                
                for i in range(3):
                    for j,c in enumerate("XYZ"):
                        idx = reorder_list[i][j]
                        pauli_strings[idx][vertex] = c

            case 2:
                predecessor0 = visited_children[0]
                predecessor1 = visited_children[1]

                #use the same reordering trick to get XXXYYYZZZ on first predecessor
                reorder_list = [[] for i in range(3)] 
                for i in range(9):
                    basis = pauli_strings[i][predecessor0]
                    reorder_list["XYZ".index(basis)].append(i)
                
                #list out string with permuted values of predecessor 2
                substring = ""
                for list in reorder_list:
                    for idx in list:
                        substring += pauli_strings[idx][predecessor1]

                #match predecessor two with a permutation of example_orderings
                reordering = ""
                for perm in permutations("XYZ"):
                    p_string = "".join(["XYZ"[perm.index(p)] for p in substring])
                    if p_string in example_orderings:
                        reordering = example_orderings[p_string]
                        break
                
                #unpermute the example orderings so that they match the original strings
                i = 0
                for list in reorder_list:
                    for idx in list:
                        pauli_strings[idx][vertex] = reordering[i]
                        i += 1

            case _: #processor needs to have connectivity so that there are <= 2 predecessors
                raise Exception("Three or more predecessors encountered")
        
        for c in children: #call recursive method on children
            if c not in visited_children:
                self._getstr(c, visited_verts)

    def _generate_gate_dict(self):
        transpiled_gates = {}
        raw_gates = {}
        backend = self.backend

        for p in "IXYZ":
            qc = QuantumCircuit(1)
            qc.append(Pauli(p),[0])
            transpiled_gates[p] = transpile(qc, basis_gates=backend._basis_gates())
            raw_gates[p] = qc

        for h,p in product(["H","HS"],"IXYZ"):
            qc = QuantumCircuit(1)
            match h:
                case "H":
                    qc.h(0)
                case "HS":
                    qc.h(0)
                    qc.s(0)
                case "I":
                    qc.id(0)
            qc.append(Pauli(p),[0])
            transpiled_gates[h+p] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[h+p] = qc

        for p,h,x in product("IXYZ", ["", "H", "SdgH"], ["","X"]):
            qc = QuantumCircuit(1)
            qc.append(Pauli(p),[0])
            match h:
                case "H":
                    qc.h(0)
                case "SdgH":
                    qc.sdg(0)
                    qc.h(0)
            if x=="X":
                qc.x(0)

            transpiled_gates[p+h+x] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[p+h+x] = qc      

        transpiled_gates['I'].id(0)
        transpiled_gates['Z'].id(0)
        transpiled_gates['XX'].id(0)

        return raw_gates, transpiled_gates

    def print_gate_conversions(self):
        raw_gates = self.raw_gates
        transpiled_gates = self.transpiled_gates

        qc = QuantumCircuit(len(raw_gates))
        for i,p in enumerate(raw_gates.keys()):
            qc = qc.compose(raw_gates[p], [i])


            qc2 = QuantumCircuit(len(transpiled_gates))

        for i,p in enumerate(transpiled_gates.keys()):
            qc2 = qc2.compose(transpiled_gates[p], [i])

        qc.barrier()
        qc = qc.compose(qc2)
        return qc

In [1312]:
from typing import NewType, Tuple

TwoQubitGate = NewType("TwoQubitGate", Tuple[str, Tuple])
#generate circuits and metadata for learning the noise in a layer of single-qubit gates
class TwoQubitGateLayerExperiment(NoiseLearningExperiment):

    '''
    layer: a list of qubit with gates contained in the layer being benchmarked
    backend: IBMQBackend to be run on 
    samples: samples from twirl to take
    depths: noise repetitions to use for exponential fit
    '''
    def __init__(self, layer, two_qubit_gates, context_qubits, backend, samples=64, depths = [2,4,8,16,32,64,128]):
        super().__init__(layer, backend, samples, depths)
        self.two_qubit_gates = two_qubit_gates
        self.context_qubits = context_qubits
        self.conjugate_gates = self._generate_conjugate_gates()

    def _generate_conjugate_gates(self):
        pauli_group = pauli_basis(2, pauli_list = True)
        pauli_dict = {p:p.to_matrix() for p in pauli_group}

        def conjugate(p, c):
            return c @ p @ np.conjugate(np.transpose(c))

        def get_name(pauli):
            for p in pauli_group:
                if Operator(pauli_dict[p]).equiv(Operator(pauli)):
                    return p

        gates_dict = {'cx':{}, 'cz':{}}
        c = CXGate().to_matrix()
        for p in pauli_group:
            gates_dict['cx'][p] = (Pauli(get_name(conjugate(pauli_dict[p],c))))

        c = CZGate().to_matrix()
        for p in pauli_group:
            gates_dict['cz'][p] = (Pauli(get_name(conjugate(pauli_dict[p],c))))

        return gates_dict

    def generate_double_instance(self, prep_basis, meas_basis, noise_repetitions, transpiled=True):

        GATE_SIZE = 2
        n = self.n
        two_qubit_gates = self.two_qubit_gates
        context_qubits = self.context_qubits
        circ = QuantumCircuit(n)

        if transpiled:
            gate_dict = self.transpiled_gates
        else: 
            gate_dict = self.raw_gates

        def without_phase(pauli):
            return Pauli((pauli.z, pauli.x))
        
        def prep(pauli_basis):
            if pauli_basis.equiv(Pauli("X")):
                return "H"
            elif pauli_basis.equiv(Pauli("Y")):
                return "HS"
            else:
                return ""

        def meas(pauli_basis):
            if pauli_basis.equiv(Pauli("X")):
                return "H"
            elif pauli_basis.equiv(Pauli("Y")):
                return "SdgH"
            else:
                return ""

        def apply(two_qubit_gate, qc):
            gate,support = two_qubit_gate
            if gate == 'cx':
                qc.cx(*support)
            if gate == 'cz':
                qc.cz(*support)

        first_twirl = choices(pauli_basis(1, pauli_list = True), k=n)

        for i,(p,b) in enumerate(zip(first_twirl, prep_basis)):
            gate_name = prep(b)
            gate_name += p.to_label()
            circ = circ.compose(gate_dict[gate_name], [i])

        if noise_repetitions > 0:
            for gate,support in two_qubit_gates:
                apply((gate,support), circ)

        circ.barrier()

        two_qubit_layers = []
        context_qubit_layers = []
        two_qubit_final_ops = []
        context_qubit_final_ops = []

        for gate,support in two_qubit_gates:

            op = first_twirl[support[1]].tensor(first_twirl[support[0]])
            op = self.conjugate_gates[gate][op]
            two_qubit_layer = []

            for j in range(noise_repetitions-1):
                twirl = choice(pauli_basis(GATE_SIZE, pauli_list = True))
                two_qubit_layer.append(twirl)
                op = without_phase(op.compose(twirl))
                op = self.conjugate_gates[gate][op]
            
            two_qubit_layers.append(two_qubit_layer)
            two_qubit_final_ops.append(op)

        for qubit in context_qubits:
            op = first_twirl[qubit]
            context_layer = []
            for i in range(noise_repetitions-1):
                twirl = choice(pauli_basis(1, pauli_list=True))
                op = without_phase(op.compose(twirl))
                context_layer.append(twirl.to_label())
            context_qubit_layers.append(context_layer)
            context_qubit_final_ops.append(op)
        
        for single_layer, double_layer in zip(list(zip(*context_qubit_layers)), list(zip(*two_qubit_layers))):

            for single_gate, qubit in zip(single_layer, context_qubits):
                circ = circ.compose(gate_dict[single_gate], [qubit])

            for double_gate, (gate, support) in zip(double_layer, two_qubit_gates):
                for p, qubit in zip(double_gate, support):
                    circ = circ.compose(gate_dict[p.to_label()], [qubit])
                apply((gate,support), circ)
            
            circ.barrier()

        rostring = "".join(choices(['I','X'], k=n))

        for op,(gate,support) in zip(two_qubit_final_ops, two_qubit_gates):
            for p,qubit in zip(op, support):
                gate_name = p.to_label()
                b = meas_basis[qubit]
                gate_name += meas(b)
                gate_name += {"X":"X", "I":""}[rostring[::-1][qubit]]
                circ = circ.compose(gate_dict[gate_name], [qubit]) 

        for op,qubit in zip(context_qubit_final_ops, context_qubits):
            gate_name = op.to_label()
            b = meas_basis[qubit]
            gate_name += meas(b)
            gate_name += {"X":"X", "I":""}[rostring[::-1][qubit]]
            circ = circ.compose(gate_dict[gate_name], [qubit])

        return circ, {"prep_basis":prep_basis, "meas_basis":meas_basis, "length":noise_repetitions, "rostring":rostring}

In [1313]:
from qiskit.providers.aer import Aer, AerSimulator
backend = Aer.get_backend('qasm_simulator')
from qiskit.providers.fake_provider import FakeQuito
backend = AerSimulator.from_backend(FakeQuito())

In [1314]:
tglp = TwoQubitGateLayerExperiment([0,1,2,3,4], [("cx",(0,1)),('cz',(3,2))], [4], backend)

[0, 1, 2, 3, 4]


In [1319]:
circ, metadata = tglp.generate_double_instance(Pauli("YYYYY"),Pauli("YYYYY"), 0, transpiled=True)

In [1320]:
circ.draw()

global phase: π/2
     ┌──────────┐┌────┐           ░   ┌────┐  ┌──────────┐           
q_0: ┤ Rz(-π/2) ├┤ √X ├───────────░───┤ √X ├──┤ Rz(-π/2) ├───────────
     ├──────────┤├────┤┌────────┐ ░ ┌─┴────┴─┐└──┬────┬──┘┌─────────┐
q_1: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π) ├─░─┤ Rz(-π) ├───┤ √X ├───┤ Rz(π/2) ├
     ├──────────┤├────┤├────────┤ ░ └─┬────┬─┘┌──┴────┴──┐└─────────┘
q_2: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π) ├─░───┤ √X ├──┤ Rz(-π/2) ├───────────
     ├─────────┬┘├────┤└────────┘ ░ ┌─┴────┴─┐└──┬────┬──┘┌─────────┐
q_3: ┤ Rz(π/2) ├─┤ √X ├───────────░─┤ Rz(-π) ├───┤ √X ├───┤ Rz(π/2) ├
     ├─────────┴┐├────┤┌────────┐ ░ └─┬────┬─┘┌──┴────┴──┐└─────────┘
q_4: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π) ├─░───┤ √X ├──┤ Rz(-π/2) ├───────────
     └──────────┘└────┘└────────┘ ░   └────┘  └──────────┘

In [1182]:
circ.measure_all()
sim = Aer.get_backend('qasm_simulator')
sim.run(circ).result().get_counts()

{'11110': 1024}

In [1291]:
metadata

{'prep_basis': Pauli('YYYYY'),
 'meas_basis': Pauli('YYYYY'),
 'length': 10,
 'rostring': 'IXXII'}

In [1197]:
tglp.print_gate_conversions().draw()

global phase: 2π
      ┌───┐                  ░    ┌───┐                            
 q_0: ┤ I ├──────────────────░────┤ I ├────────────────────────────
      ├───┤                  ░    ├───┤                            
 q_1: ┤ X ├──────────────────░────┤ X ├────────────────────────────
      ├───┤                  ░  ┌─┴───┴──┐    ┌───┐                
 q_2: ┤ Y ├──────────────────░──┤ Rz(-π) ├────┤ X ├────────────────
      ├───┤                  ░  ├───────┬┘    ├───┤                
 q_3: ┤ Z ├──────────────────░──┤ Rz(π) ├─────┤ I ├────────────────
      ├───┤ ┌───┐            ░ ┌┴───────┴┐    ├───┴┐   ┌─────────┐ 
 q_4: ┤ H ├─┤ I ├────────────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├─
      ├───┤ ├───┤            ░ ├─────────┴┐   ├────┤   ├─────────┤ 
 q_5: ┤ H ├─┤ X ├────────────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├─
      ├───┤ ├───┤            ░ ├──────────┤   ├────┤   ├─────────┴┐
 q_6: ┤ H ├─┤ Y ├────────────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├
      ├───┤ ├───┤            ░ ├─────────┬┘   ├────┤   ├──────────┤
 q_7: ┤ H ├─┤ Z ├────────────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├───┤ ├───┤ ┌───┐      ░ ├─────────┤    ├────┤   └┬────────┬┘
 q_8: ┤ H ├─┤ S ├─┤ I ├──────░─┤ Rz(π/2) ├────┤ √X ├────┤ Rz(-π) ├─
      ├───┤ ├───┤ ├───┤      ░ ├─────────┴┐   ├────┤    └────────┘ 
 q_9: ┤ H ├─┤ S ├─┤ X ├──────░─┤ Rz(-π/2) ├───┤ √X ├───────────────
      ├───┤ ├───┤ ├───┤      ░ ├──────────┤   ├────┤    ┌────────┐ 
q_10: ┤ H ├─┤ S ├─┤ Y ├──────░─┤ Rz(-π/2) ├───┤ √X ├────┤ Rz(-π) ├─
      ├───┤ ├───┤ ├───┤      ░ ├─────────┬┘   ├────┤    └────────┘ 
q_11: ┤ H ├─┤ S ├─┤ Z ├──────░─┤ Rz(π/2) ├────┤ √X ├───────────────
      ├───┤ ├───┤ └───┘      ░ └──┬───┬──┘    └────┘               
q_12: ┤ I ├─┤ X ├────────────░────┤ X ├────────────────────────────
      ├───┤ ├───┤            ░ ┌──┴───┴──┐    ┌────┐   ┌─────────┐ 
q_13: ┤ I ├─┤ H ├────────────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├─
      ├───┤ ├───┤ ┌───┐      ░ ├─────────┴┐   ├────┤   ├─────────┤ 
q_14: ┤ I ├─┤ H ├─┤ X ├──────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├─
      ├───┤┌┴───┴┐├───┤      ░ └──┬────┬──┘┌──┴────┴─┐ └─────────┘ 
q_15: ┤ I ├┤ Sdg ├┤ H ├──────░────┤ √X ├───┤ Rz(π/2) ├─────────────
      ├───┤├─────┤├───┤┌───┐ ░  ┌─┴────┴─┐ └──┬────┬─┘ ┌─────────┐ 
q_16: ┤ I ├┤ Sdg ├┤ H ├┤ X ├─░──┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├─
      ├───┤└┬───┬┘└───┘└───┘ ░  └─┬───┬──┘    └────┘   └─────────┘ 
q_17: ┤ X ├─┤ X ├────────────░────┤ I ├────────────────────────────
      ├───┤ ├───┤            ░ ┌──┴───┴──┐    ┌────┐   ┌──────────┐
q_18: ┤ X ├─┤ H ├────────────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├───┤ ├───┤ ┌───┐      ░ ├─────────┴┐   ├────┤   ├──────────┤
q_19: ┤ X ├─┤ H ├─┤ X ├──────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├
      ├───┤┌┴───┴┐├───┤      ░ └┬────────┬┘   ├────┤   ├──────────┤
q_20: ┤ X ├┤ Sdg ├┤ H ├──────░──┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├───┤├─────┤├───┤┌───┐ ░  └─┬────┬─┘ ┌──┴────┴──┐└──────────┘
q_21: ┤ X ├┤ Sdg ├┤ H ├┤ X ├─░────┤ √X ├───┤ Rz(-π/2) ├────────────
      ├───┤└┬───┬┘└───┘└───┘ ░  ┌─┴────┴─┐ └──────────┘            
q_22: ┤ Y ├─┤ X ├────────────░──┤ Rz(-π) ├─────────────────────────
      ├───┤ ├───┤            ░ ┌┴────────┴┐   ┌────┐   ┌──────────┐
q_23: ┤ Y ├─┤ H ├────────────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├
      ├───┤ ├───┤ ┌───┐      ░ ├─────────┬┘   ├────┤   ├──────────┤
q_24: ┤ Y ├─┤ H ├─┤ X ├──────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├───┤┌┴───┴┐├───┤      ░ └──┬────┬─┘ ┌──┴────┴──┐└──────────┘
q_25: ┤ Y ├┤ Sdg ├┤ H ├──────░────┤ √X ├───┤ Rz(-π/2) ├────────────
      ├───┤├─────┤├───┤┌───┐ ░  ┌─┴────┴─┐ └──┬────┬──┘┌──────────┐
q_26: ┤ Y ├┤ Sdg ├┤ H ├┤ X ├─░──┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├───┤└┬───┬┘└───┘└───┘ ░  ├───────┬┘    ├───┬┘   └──────────┘
q_27: ┤ Z ├─┤ X ├────────────░──┤ Rz(π) ├─────┤ X ├────────────────
      ├───┤ ├───┤            ░ ┌┴───────┴─┐   ├───┴┐   ┌─────────┐ 
q_28: ┤ Z ├─┤ H ├────────────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├─
      ├───┤ ├───┤ ┌───┐      ░ ├───────